In [2]:
import pandas as pd

df=pd.read_excel("ramen-ratings.xlsx")
features = ["Brand", "Variety", "Style", "Country", "Stars", "Spicy?", "Main Flavor"]
df = df[features].dropna()
df['Variety'] = df['Variety'].astype(str)
df_encoded = pd.get_dummies(df, columns = ["Brand", "Style", "Country", "Spicy?", "Main Flavor"])
df_encoded_clean = df_encoded.drop(columns=["Variety", "Stars"])
df_encoded.head()


,Variety,Stars,Brand_1 To 3 Noodles,Brand_7 Select,Brand_7 Select/Nissin,Brand_A-One,Brand_A-Sha Dry Noodle,Brand_A1,Brand_ABC,Brand_Acecook,...,Main Flavor_Beef,Main Flavor_Chicken,Main Flavor_Crab,Main Flavor_Kimchi,Main Flavor_Miso,Main Flavor_Mushroom,Main Flavor_Others,Main Flavor_Pork,Main Flavor_Shrimp,Main Flavor_Vegetable
0,T's Restaurant Tantanmen,3.75,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,Noodles Spicy Hot Sesame Spicy Hot Sesame Guan...,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,Cup Noodles Chicken Vegetable,2.25,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,GGE Ramen Snack Tomato Flavor,2.75,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,Singapore Curry,3.75,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [4]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

knn = NearestNeighbors(n_neighbors = 5, metric = 'cosine')
knn.fit(df_encoded_clean)

def recommend (variety, n=5, spicy=False):
    print(f"Searching for: {variety}")
    df['Variety'] = df['Variety'].str.strip()
   
    item_index = df[df['Variety'].str.strip().str.contains(variety, case=False)].index
    if (spicy==True):
        item_index = df[(df['Variety'].str.strip().str.contains(variety, case=False) & df['Spicy?'] == True)].index
    if len(item_index) == 0:
        return "No recommendations found. Please try again."
    item_vector = df_encoded_clean.iloc[item_index].values
   
    if item_vector.shape[0] == 0:
        return "Error: The item vector is empty."
    distances, indices = knn.kneighbors(item_vector, n_neighbors = n+1)

    recommendations = df.iloc[indices[0][1:]]
    print(f"Here are some recommendations if you want {variety} :")
    return recommendations[["Brand", "Variety", "Style", "Country", "Stars", "Spicy?", "Main Flavor"]]



In [23]:
recommend("buldak", n=5, spicy = True)

Searching for: buldak
Here are some recommendations if you want buldak :


D:\conda\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Brand,Variety,Style,Country,Stars,Spicy?,Main Flavor
213,Samyang Foods,2x Spicy Haek Buldak Bokkeummyun,Pack,South Korea,4,True,Others
1082,Samyang Foods,Sogokimyun Hot Flavor Noodle Soup,Pack,South Korea,3.5,True,Others
2560,Nongshim,Neoguri (Seafood'n'Spicy),Pack,South Korea,3.5,True,Others
2564,Samyang,Kalgug-Su (Spicy),Pack,South Korea,3.5,True,Others
2562,Ottogi,Jin Ramen (Hot Taste),Pack,South Korea,3.5,True,Others
